In [0]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Flatten
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
import keras.backend as K

BATCH_SIZE = 4

Using TensorFlow backend.


___
# Normalize data #
___

In [0]:
def normalize(image):
    image = tf.cast(image, tf.float32) * (1.0 / 255.0) - 0.5
    return image

___
# Read .tfrecords file and get data #
___

In [0]:
def read_and_decode(filename_queue):
    
    files = tf.data.Dataset.list_files(filename_queue)
    dataset = files.interleave(tf.data.TFRecordDataset,cycle_length=1)
    def parser(record):
        keys_to_features = {
            'old_frame': tf.io.FixedLenFeature([], tf.string, default_value=""),
            'new_frame': tf.io.FixedLenFeature([], tf.string, default_value=""),
            'old_feature_point': tf.io.VarLenFeature( tf.int64),
            'new_feature_point':  tf.io.VarLenFeature( tf.int64),
            'points_shape':  tf.io.FixedLenFeature([], tf.int64,default_value=0),
            'x_mesh': tf.io.VarLenFeature( tf.float32),
            'y_mesh': tf.io.VarLenFeature( tf.float32),
        }
        parsed = tf.parse_single_example(record, keys_to_features)
        old_frame_parse = tf.decode_raw(parsed['old_frame'], tf.uint8)
        old_frame_parse = tf.reshape(old_frame_parse, [288, 512, 3])
        old_frame_parse.set_shape( [288, 512, 3])
        new_frame_parse = tf.decode_raw(parsed['new_frame'], tf.uint8)
        new_frame_parse = tf.reshape(new_frame_parse,  [288, 512, 3])
        new_frame_parse.set_shape( [288, 512, 3])
        
        #normalize images
        old_frame_parse = normalize(old_frame_parse)
        new_frame_parse = normalize(new_frame_parse)
        input_frames = tf.concat([old_frame_parse,new_frame_parse],1)
        ##get feature points shape
        #points_shape = tf.cast(parsed['points_shape'], tf.int32)
        ##load all feature point of old & new frame
        #old_feature_point = tf.cast(tf.sparse.to_dense(parsed['old_feature_point'],default_value = 0),tf.int32)
        #old_feature_point = tf.reshape(old_feature_point,[points_shape,2])
        #new_feature_point = tf.cast(tf.sparse.to_dense(parsed['new_feature_point'],default_value = 0),tf.int32)
        #new_feature_point = tf.reshape(new_feature_point,[points_shape,2])
        #load all mesh
        #Bắt đầu từ dataset thứ 11 phải chuẩn hóa lại /255 - 0.5
        x_mesh = tf.cast(tf.sparse.to_dense(parsed['x_mesh'],default_value = 0),tf.float32)
        x_mesh = tf.reshape(x_mesh,[19, 33])
        x_mesh = tf.reshape(x_mesh,[19*33])
        y_mesh = tf.cast(tf.sparse.to_dense(parsed['y_mesh'],default_value = 0),tf.float32)
        y_mesh = tf.reshape(y_mesh,[19, 33])
        y_mesh = tf.reshape(y_mesh,[19*33])
        groundtruth_mesh = tf.concat([x_mesh,y_mesh],0)
        return input_frames,groundtruth_mesh#,old_feature_point, new_feature_point
    
    #train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(buffer_size=2000)
    dataset = dataset.padded_batch(BATCH_SIZE,drop_remainder=True,
                                  padded_shapes=([288, 1024, 3],
                                                 [19*33*2]))
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    input_frames,groundtruth_mesh = iterator.get_next()
    
    
    return input_frames,groundtruth_mesh#,old_feature_point, new_feature_point

___
# Loss function #
___

## Custom loss 1##

In [0]:
def custom_loss(neighbor_matrix, batch_size):
  def Loss(y_true, y_pred):
    ## Groundtruth loss
    #groundtruth_loss = tf.math.subtract(y_true,y_pred)
    #groundtruth_loss = tf.math.square(groundtruth_loss)
    #groundtruth_loss = tf.math.reduce_sum(groundtruth_loss,1)
    #groundtruth_loss = tf.math.reduce_mean(tf.math.sqrt(groundtruth_loss))/1254.0
    
    similar_loss = tf.keras.losses.hinge(y_true, y_pred)
    ## Similarity loss
    ##create prediction matrix size of (batch, max_neighbor, 1254) 
    #y = tf.identity(y_pred)
    #y = tf.reshape(y,[batch_size,1254,1])
    #y = tf.tile(y,[1,1,8])
    #
    #neighbor_index_matrix = tf.reshape(neighbor_matrix,(1,1254,8))
    #neighbor_index_matrix = tf.tile(neighbor_index_matrix,[batch_size,1,1])
    #neighbor_index_matrix = tf.reshape(neighbor_index_matrix,
    #                                   (batch_size,1254,8,1))
    #
    #batch_index = tf.range(0, batch_size)
    #batch_index = tf.reshape(batch_index,[batch_size,1])
    #batch_index = tf.tile(batch_index,[1,8])
    #batch_index = tf.reshape(batch_index,[batch_size,1,8])
    #batch_index = tf.tile(batch_index,[1,1254,1])
    #
    #batch_index = tf.reshape(batch_index,[batch_size,1254,8,1])
    #neighbor_index_matrix = tf.cast(neighbor_index_matrix,tf.int32)
    #neighbor_index_matrix = tf.concat([batch_index,neighbor_index_matrix],3)
    #neighbor_predict_matrix = tf.gather_nd(y_pred,neighbor_index_matrix)
    #
    #x_split, y_split = tf.split(neighbor_predict_matrix,2,1)
    #
    #similar_loss = tf.math.square(tf.math.subtract(x_split,x_predict)) +\
    #                tf.math.square(tf.math.subtract(y_split,y_predict))
    #similar_loss = tf.math.sqrt(tf.math.reduce_sum(similar_loss,2))/8.0
    ##similar_loss will be shape [4,1254], we still need to reshape it to
    ##fit the shape of groundtruth loss
    #
    #similar_loss = tf.math.reduce_mean(similar_loss)
    
    # Feature loss
    
    #num_feature_points = tf.shape(old_feature_point)[1]
    #  #old_point
    #old_points = tf.reshape(old_feature_point,
    #                        [batch_size,1,num_feature_points,2])
    #old_points = tf.tile(old_points,[1,9,1,1])
    #old_points = tf.cast(old_points,tf.float32)
    #  #new_point
    #new_points = tf.reshape(new_feature_point,
    #                        [batch_size,1,num_feature_points,2])
    #
    #new_points = tf.tile(new_points,[1,9,1,1])
    #new_points = tf.cast(new_points,tf.float32)
    #  #get motion
    #motion = tf.identity(y_pred)
    #x_original_meshes = tf.slice(motion,[0,0],[batch_size,627])
    #y_original_meshes = tf.slice(motion,[0,627],[batch_size,627])
    #
    #downsample_meshes = tf.constant([0,16,32,
    #                                   9*33,9*33+16,9*33+32,
    #                                   18*33,18*33+16,18*33+32])
    #downsample_meshes = tf.reshape(downsample_meshes,[1,9])
    #downsample_meshes = tf.tile(downsample_meshes,[batch_size,1])
    #downsample_meshes = tf.reshape(downsample_meshes,[batch_size,9,1])
    #
    #batch_index = tf.range(0, batch_size)
    #batch_index = tf.reshape(batch_index,[batch_size,1])
    #batch_index = tf.tile(batch_index,[1,9])
    #batch_index = tf.reshape(batch_index,[batch_size,9,1])
    #
    #downsample_meshes = tf.concat([batch_index,downsample_meshes],2)
    #x_downsample_motion_meshes =tf.gather_nd(x_original_meshes,
    #                                         downsample_meshes)/512.0
    #y_downsample_motion_meshes = tf.gather_nd(y_original_meshes,
    #                                          downsample_meshes)/288.0
    #x_downsample_motion_meshes = tf.reshape(x_downsample_motion_meshes,
    #                                        [batch_size,9,1])
    #y_downsample_motion_meshes = tf.reshape(y_downsample_motion_meshes,
    #                                        [batch_size,9,1])
    #motion = tf.concat([x_downsample_motion_meshes, y_downsample_motion_meshes],
    #                   2)
    #motion = tf.reshape(motion,[batch_size,9,1,2])
    #motion = tf.tile(motion,[1,1,num_feature_points,1])
    #pred_location = tf.multiply(old_points,(motion+1.0))
    #
    #feature_loss = tf.math.subtract(pred_location,new_points)
    #feature_loss = tf.math.square(feature_loss)
    #feature_loss = tf.math.reduce_sum(feature_loss,axis=3)
    #feature_loss = tf.math.sqrt(tf.math.reduce_sum(feature_loss))\
    #                /tf.cast(num_feature_points,tf.float32) 
    ##shape (batch,9)
    #feature_loss = tf.math.reduce_mean(feature_loss)
    # similar_loss ++feature_loss
    return   similar_loss #+ 50*groundtruth_loss#+feature_loss
  return Loss

# Learning rate schedule #

In [0]:
NUM_ITERATION_PER_EPOCH = 100*2*2

def schedule(epoch, lr):
  iteration = epoch*NUM_ITERATION_PER_EPOCH
  decay_step = 30000
  if iteration /decay_step >= 1 and (iteration-(iteration%decay_step)
                                     *decay_step - NUM_ITERATION_PER_EPOCH) <0:
        return lr*0.1
  else:
    return lr
lrate = tf.keras.callbacks.LearningRateScheduler(schedule,verbose = 1)

In [0]:
def Create_model():
  base_model = ResNet50(weights= 'imagenet', include_top=False, 
                        input_shape= (288,1024,3))
  
  model = Sequential([base_model,
                      Flatten(),
                     Dense(256,activation='relu'),
                      Dense(128,activation='relu'),
                      #Dense(512,activation='relu'),
                     Dense(19*33*2,activation="linear")])
  return model

___
# Main procedure #
___

## Load training and validation data ##

In [0]:
input_frames,groundtruth_mesh\
= read_and_decode('./gdrive/My Drive/Dataset3/Train/*.tfrecords')
validate_input_frames,validate_groundtruth_mesh\
= read_and_decode('./gdrive/My Drive/Dataset3/Validate/*.tfrecords')

W0701 16:16:51.232580 140165084358528 deprecation.py:323] From <ipython-input-4-67ede45fdc95>:52: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


## Initial training ##

In [0]:
TRAIN_STEPS = 100*2*2
VALIDATION_STEPS = 20*2*2
checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath='./gdrive/My Drive/model.h5',
        save_best_only=True,
        monitor='val_loss',
        verbose=1)

#Combine callbacks
callbacks_list = [checkpoint,lrate]
opt=tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9,beta_2=0.999)
model = Create_model()
model.compile(optimizer=opt,
              loss=custom_loss(Get_Neighbor_Matrix(), BATCH_SIZE))
#,old_feature_point,new_feature_point
model.summary()
model.fit(input_frames, groundtruth_mesh,
          batch_size = BATCH_SIZE,
          epochs=1,
          callbacks=callbacks_list,
          validation_steps = VALIDATION_STEPS,
          validation_data = (validate_input_frames,validate_groundtruth_mesh),
          steps_per_epoch = TRAIN_STEPS)

tf.keras.backend.clear_session()
#del X
gc.collect()
#model.save_weights('./gdrive/My Drive/model.h5')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W0701 16:17:33.444319 140165084358528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 9, 32, 2048)       23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 589824)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               150995200 
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1254)              161766    
Total params: 174,777,574
Trainable params: 174,724,454
Non-trainable params: 53,120
_________________________________________________________________

Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
399/400 [=======

## Cotinue train from save weight ##

In [0]:
TRAIN_STEPS = 100*2*2
VALIDATION_STEPS = 20*2*2
checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath='./gdrive/My Drive/model13.h5',
        save_best_only=True,
        monitor='val_loss',
        verbose=1)

#Combine callbacks
callbacks_list = [checkpoint,lrate]
opt=tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9,beta_2=0.999)
input_frames1,groundtruth_mesh1\
= read_and_decode('./gdrive/My Drive/Dataset3/Train13/*.tfrecords')
validate_input_frames1,validate_groundtruth_mesh1\
= read_and_decode('./gdrive/My Drive/Dataset3/Validate13/*.tfrecords')

weights_path = './gdrive/My Drive/model12.h5'
model1 = Create_model()
model1.load_weights(weights_path)
model1.compile(optimizer=opt,
              loss=custom_loss(Get_Neighbor_Matrix(), 
              BATCH_SIZE))
model1.fit(input_frames1, groundtruth_mesh1,
          batch_size = BATCH_SIZE,
          epochs=1,
          callbacks=callbacks_list,
           validation_steps = VALIDATION_STEPS,
          validation_data = (validate_input_frames1,validate_groundtruth_mesh1),
          steps_per_epoch = TRAIN_STEPS)#,validation_split=0.2


W0701 16:13:50.092340 140005571860352 deprecation.py:323] From <ipython-input-4-67ede45fdc95>:52: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W0701 16:13:58.103060 140005571860352 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future vers


Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-06.
280/400 [====================>.........] - ETA: 50s - loss: 0.9566